# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [26]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,anadyr,64.7500,177.4833,267.43,79,100,4.00,RU,1730579344
1,1,puerto aysen,-45.4000,-72.7000,281.99,97,99,2.41,CL,1730579345
2,2,buka,40.8108,69.1986,286.53,47,0,2.43,UZ,1730579346
3,3,la passe,45.5549,-0.8967,287.09,87,0,3.79,FR,1730579348
4,4,ust-nera,64.5667,143.2000,248.86,100,95,2.07,RU,1730579349


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [16]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot_3 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    scale = 1,
    color = "City"
)


# Display the map
map_plot_3

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [139]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
ideal_weather = city_data_df.loc[(city_data_df['Cloudiness'] == 0) & (city_data_df['Wind Speed'] < 4.5)
                                        & (city_data_df['Max Temp'] < 300.14) & (city_data_df['Max Temp'] > 294.15) ] 


# Drop any rows with null values
# YOUR CODE HERE
#ideal_weather = ideal_weather.append(city_data_df.loc[(city_data_df['City_ID'] == 291)])
# Display sample data
ideal_weather

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
13,13,nova sintra,14.8667,-24.7167,299.89,77,0,1.65,CV,1730579360
63,63,selebi-phikwe,-21.9790,27.8498,297.75,30,0,2.59,BW,1730579421
113,113,bam,29.1060,58.3570,294.75,27,0,0.94,IR,1730579480
174,174,veraval,20.9000,70.3667,299.92,54,0,2.15,IN,1730579554
232,232,khambhaliya,22.2072,69.6683,299.07,65,0,2.65,IN,1730579625
261,261,minab,27.1467,57.0801,299.93,62,0,0.29,IR,1730579658
377,377,huntsville,34.7304,-86.5861,299.04,68,0,3.09,US,1730579797
380,380,sabha,27.0377,14.4283,294.25,38,0,2.69,LY,1730579801
399,399,badiyah,22.4500,58.8000,297.82,39,0,0.46,OM,1730579825
427,427,faridpur,23.6000,89.8333,297.43,75,0,2.16,BD,1730579859


### Step 3: Create a new DataFrame called `hotel_df`.

In [141]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = ideal_weather[['City_ID','City','Country','Lat','Lng','Humidity']].copy()

hotel_df['Hotel Name'] = ''
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE

# Display sample data
hotel_df

,City_ID,City,Country,Lat,Lng,Humidity,Hotel Name
13,13,nova sintra,CV,14.8667,-24.7167,77,
63,63,selebi-phikwe,BW,-21.9790,27.8498,30,
113,113,bam,IR,29.1060,58.3570,27,
174,174,veraval,IN,20.9000,70.3667,54,
232,232,khambhaliya,IN,22.2072,69.6683,65,
261,261,minab,IR,27.1467,57.0801,62,
377,377,huntsville,US,34.7304,-86.5861,68,
380,380,sabha,LY,27.0377,14.4283,38,
399,399,badiyah,OM,22.4500,58.8000,39,
427,427,faridpur,BD,23.6000,89.8333,75,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [143]:
# Set parameters to search for a hotel
radius = 10000
filters = f"circle:{''},{''},{''}"
bias = f"proximity:{''},{''}"
categories = "accommodation.hotel"

params = {
    "categories":categories,
    "filter":filters,
    "bias":bias,
    "apiKey":geoapify_key    
}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Set base URL
base_url = "https://api.geoapify.com/v2/places"

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    longitude = hotel_df.loc[index, "Lng"]
    latitude = hotel_df.loc[index, "Lat"]
    
    filters = f"circle:{longitude},{latitude},{radius}"
    bias = f"proximity:{longitude},{latitude}"
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] =filters
    
    params["bias"] = bias

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
nova sintra - nearest hotel: Residência Ka Dencho
selebi-phikwe - nearest hotel: Cresta Bosele Hotel
bam - nearest hotel: هتل جهانگردی بم
veraval - nearest hotel: Shyam
khambhaliya - nearest hotel: No hotel found
minab - nearest hotel: هتل صدف
huntsville - nearest hotel: 106 Jefferson Huntsville, Curio Collection by Hilton
sabha - nearest hotel: فندق الكاظم-سبها
badiyah - nearest hotel: Al Areen Hotel Apartments
faridpur - nearest hotel: Hotel Reverse Inn
mulayjah - nearest hotel: No hotel found
toliara - nearest hotel: Ambary
saint-pierre - nearest hotel: Tropic Hotel


,City_ID,City,Country,Lat,Lng,Humidity,Hotel Name
13,13,nova sintra,CV,14.8667,-24.7167,77,Residência Ka Dencho
63,63,selebi-phikwe,BW,-21.9790,27.8498,30,Cresta Bosele Hotel
113,113,bam,IR,29.1060,58.3570,27,هتل جهانگردی بم
174,174,veraval,IN,20.9000,70.3667,54,Shyam
232,232,khambhaliya,IN,22.2072,69.6683,65,No hotel found
261,261,minab,IR,27.1467,57.0801,62,هتل صدف
377,377,huntsville,US,34.7304,-86.5861,68,"106 Jefferson Huntsville, Curio Collection by ..."
380,380,sabha,LY,27.0377,14.4283,38,فندق الكاظم-سبها
399,399,badiyah,OM,22.4500,58.8000,39,Al Areen Hotel Apartments
427,427,faridpur,BD,23.6000,89.8333,75,Hotel Reverse Inn


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [151]:
%%capture --no-display

# Configure the map plot
map_plot_4 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    scale = 0.5,
    color = "City",
    frame_width = 800,
    frame_height = 600,  
    hover_cols=['Hotel Name', 'Country']
    
)


# Display the map
map_plot_4

# Display the map
# YOUR CODE HERE

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)